# Connect

In [1]:
# Panda hostname/IP and Desk login information of your robot
hostname = '172.16.0.2'
username = 'admin'
password = 'wanglab123'

# panda-py is chatty, activate information log level
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import panda_py

desk = panda_py.Desk(hostname, username, password, platform='fr3')
desk.unlock()
desk.activate_fci()

INFO:desk:Login succesful.
INFO:desk:Retaken control.


In [3]:
from panda_py import libfranka

panda = panda_py.Panda(hostname)
gripper = libfranka.Gripper(hostname)

INFO:panda:Connected to robot (172.16.0.2).


# Eye-hand Calib

In [4]:
import numpy as np

def process_object(args):
    # Unpack the list
    score = args[0]
    width = args[1]
    height = args[2]
    depth = args[3]
    rotation_matrix = np.array(args[4:13]).reshape(3, 3)  # Reshape the rotation matrix
    translation = np.array(args[13:16])  # Extract the translation vector
    object_id = args[16]

    # Do something with the object parameters
    print("Object parameters:")
    print("Score:", score)
    print("Dimensions (width, height, depth):", width, height, depth)
    print("Rotation Matrix:\n", rotation_matrix)
    print("Translation:", translation)
    print("Object ID:", object_id)
    
    return score, width, height, depth, rotation_matrix, translation, object_id

# Example usage
args_list = [0.14745911955833435, 0.05920447036623955, 0.019999999552965164, 0.019999999552965164, 0.2812020480632782, 0.011644020676612854, -0.9595779180526733, 0.07559296488761902, -0.9970880746841431, 0.010053157806396484, -0.9566666483879089, -0.07536430656909943, -0.28126341104507446, 0.5924999713897705, 0.0925000011920929, 0.040629517287015915, -1]


score, width, height, depth, rotation_matrix, translation, object_id = process_object(args_list)


Object parameters:
Score: 0.14745911955833435
Dimensions (width, height, depth): 0.05920447036623955 0.019999999552965164 0.019999999552965164
Rotation Matrix:
 [[ 0.28120205  0.01164402 -0.95957792]
 [ 0.07559296 -0.99708807  0.01005316]
 [-0.95666665 -0.07536431 -0.28126341]]
Translation: [0.59249997 0.0925     0.04062952]
Object ID: -1


In [5]:
import roboticstoolbox as rtb
robot = rtb.models.Panda()
from spatialmath import SE3
import transforms3d.euler as euler
roll, pitch, yaw = euler.mat2euler(rotation_matrix)


Tep = SE3.Trans(0.59249997, 0.0925, 0.04062952) * SE3.RPY([roll, pitch, yaw]) * SE3.RPY([np.pi/2, 0, np.pi/2])
print(Tep)
sol = robot.ik_LM(Tep)         # solve IK
print(sol)

   0.01164  -0.9596    0.2812    0.5925    
  -0.9971    0.01005   0.07559   0.0925    
  -0.07536  -0.2813   -0.9567    0.04063   
   0         0         0         1         

(array([ 0.6459171 ,  0.66453763, -0.39538289, -2.09710101,  1.07052517,
        2.90157166,  1.69325127]), 1, 39, 4, 3.887130437448611e-08)


In [6]:
import numpy as np
panda.move_to_start()
gripper.move(0.08, 0.2)
panda.move_to_joint_position(sol[0])


pose = panda.get_pose()
print(pose)
gripper.grasp(0, 0.2, 10, 0.04, 0.04)
panda.move_to_start()
gripper.move(0.08, 0.2)

INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 0.08 seconds.
INFO:panda:Starting new controller (Trajectory).
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 3.62 seconds.
INFO:panda:Starting new controller (Trajectory).


[[ 2.16468871e-02 -9.61394925e-01  2.74287085e-01  5.91280297e-01]
 [-9.96886776e-01  1.38697507e-05  7.87249808e-02  9.39524170e-02]
 [-7.56896013e-02 -2.75137318e-01 -9.58420040e-01  3.89490793e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 3.63 seconds.
INFO:panda:Starting new controller (Trajectory).


True